This notebook is my first exploration of the RGI-mapped BN10 genomes to the CARD databases. I used all default settings with RGI, and grabbed only the hits which were deemed "strict" (i.e. 100% match). Let's see what's there!

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
fname = '../../data/clean/bn10.rgi_mapped.strict.txt'

cols = ['ORF_ID', 'CONTIG', 'START', 'STOP', 'ORIENTATION',
        'CUT_OFF', 'PASS_EVALUE', 'Best_Hit_evalue',
        'Best_Hit_ARO', 'Best_Identities', 'ARO', 'ARO_name',
        'Model_type', 'SNP', 'Best_Hit_ARO_category',
        'ARO_category', 'PASS_bitscore', 'Best_Hit_bitscore',
        'bit_score', 'Predicted_DNA', 'Predicted_Protein',
        'CARD_Protein_Sequence', 'LABEL', 'ID', 'Model_id',
        'genome']
cols = [c.lower() for c in cols]
df = pd.read_csv(fname, sep='\t', names=cols)
df.head()

,orf_id,contig,start,stop,orientation,cut_off,pass_evalue,best_hit_evalue,best_hit_aro,best_identities,...,pass_bitscore,best_hit_bitscore,bit_score,predicted_dna,predicted_protein,card_protein_sequence,label,id,model_id,genome
0,scaffold57|size36017_15,ID=57_15;partial=00;start_type=ATG;rbs_motif=T...,21782,23707,+,Strict,NaN,0.0,tetQ,96.41,...,1200,1281.540,1281.54,ATGAATATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAA...,MNIINLGILAHIDAGKTSVTENLLFASGATEKCGRVDNGDTITDSM...,MRFDNASIVVYYCLIQMNIINLGILAHIDAGKTSVTENLLFASGAT...,scaffold57|size36017_15#21783#23708#1#ID=57_15...,gnl|BL_ORD_ID|2201|hsp_num:0,1183,aa_0143_0005_c5
1,scaffold238|size8236_1,ID=238_1;partial=00;start_type=ATG;rbs_motif=N...,1445,2245,-,Strict,NaN,0.0,ErmF,98.50,...,400,531.561,531.561,ATGACAAAAAAGAAATTGCCCGTTCGTTTTACGGGTCAGCACTTTA...,MTKKKLPVRFTGQHFTIDKVLIKDAIRQANISNQDTVLDIGAGKGF...,MTKKKLPVRFTGQHFTIDKVLIKDAIRQANISNQDTVLDIGAGKGF...,scaffold238|size8236_1#1446#2246#-1#ID=238_1;p...,gnl|BL_ORD_ID|159|hsp_num:0,621,aa_0143_0005_c5
2,scaffold44|size37419_2,ID=44_2;partial=00;start_type=ATG;rbs_motif=No...,1108,2274,-,Strict,NaN,0.0,tetX,99.74,...,750,799.660,799.66,ATGACAATGCGAATAGATACAGACAAACAAATGAATTTACTTAGTG...,MTMRIDTDKQMNLLSDKNVAIIGGGPVGLTMAKLLQQNGIDVSVYE...,MTMRIDTDKQMNLLSDKNVAIIGGGPVGLTMAKLLQQNGIDVSVYE...,scaffold44|size37419_2#1109#2275#-1#ID=44_2;pa...,gnl|BL_ORD_ID|1398|hsp_num:0,776,cx_0004_0077_d9
3,scaffold44|size37419_1,ID=44_1;partial=00;start_type=ATG;rbs_motif=No...,95,895,+,Strict,NaN,0.0,ErmF,99.25,...,400,534.643,534.643,ATGACAAAAAAGAAATTGCCCGTTCGTTTTACGGGTCAGCACTTTA...,MTKKKLPVRFTGQHFTIDKVLIKDAIRQANISNQDTVLDIGAGKGF...,MTKKKLPVRFTGQHFTIDKVLIKDAIRQANISNQDTVLDIGAGKGF...,scaffold44|size37419_1#96#896#1#ID=44_1;partia...,gnl|BL_ORD_ID|159|hsp_num:0,621,cx_0004_0077_d9
4,scaffold13|size144083_82,ID=13_82;partial=00;start_type=ATG;rbs_motif=N...,110871,112796,+,Strict,NaN,0.0,tetQ,96.41,...,1200,1281.540,1281.54,ATGAATATTATAAATTTAGGAATTCTTGCTCACATTGATGCAGGAA...,MNIINLGILAHIDAGKTSVTENLLFASGATEKCGRVDNGDTITDSM...,MRFDNASIVVYYCLIQMNIINLGILAHIDAGKTSVTENLLFASGAT...,scaffold13|size144083_82#110872#112797#1#ID=13...,gnl|BL_ORD_ID|2201|hsp_num:0,1183,cx_0004_0077_d9


In [12]:
fmeta = '../../data/raw/table_BN10_WGS_metadata_03282018.txt'
# Not sure what hte last four columns are, guess for now
cols = ['genome', 'taxa', 'nan', 'donor', 'time_point', 'plate_id', 'well_id']
meta = pd.read_csv(fmeta, sep='\t', names=cols)
# One of the columns is all NaN
meta = meta.dropna(how='all', axis=1)
meta.head()

,genome,taxa,donor,genome_id,day,media
0,am_0224_0086_a4,Akkermansia_muciniphila,am,224,86,a4
1,am_0171_0066_c8,Akkermansia_muciniphila,am,171,66,c8
2,am_0171_0071_c9,Akkermansia_muciniphila,am,171,71,c9
3,am_0224_0086_h4,Akkermansia_muciniphila,am,224,86,h4
4,am_0014_0091_a10,Akkermansia_muciniphila,am,14,91,a10


In [14]:
# Merge on common genome name
print(df.shape, meta.shape)
df = pd.merge(df, meta)
print(df.shape)

((19315, 26), (3707, 6))
(19315, 31)


In [23]:
# Count number of hits per genome
hits = df.groupby('genome').size().reset_index()
hits.columns = ['genome', 'n_hits']

resdf = pd.merge(df, hits, how='left')
print(resdf.shape)
resdf.head()

(19315, 32)


,orf_id,contig,start,stop,orientation,cut_off,pass_evalue,best_hit_evalue,best_hit_aro,best_identities,...,label,id,model_id,genome,taxa,donor,genome_id,day,media,n_hits
0,scaffold57|size36017_15,ID=57_15;partial=00;start_type=ATG;rbs_motif=T...,21782,23707,+,Strict,NaN,0.0,tetQ,96.41,...,scaffold57|size36017_15#21783#23708#1#ID=57_15...,gnl|BL_ORD_ID|2201|hsp_num:0,1183,aa_0143_0005_c5,Bacteroides_ovatus,aa,143,5,c5,2
1,scaffold238|size8236_1,ID=238_1;partial=00;start_type=ATG;rbs_motif=N...,1445,2245,-,Strict,NaN,0.0,ErmF,98.50,...,scaffold238|size8236_1#1446#2246#-1#ID=238_1;p...,gnl|BL_ORD_ID|159|hsp_num:0,621,aa_0143_0005_c5,Bacteroides_ovatus,aa,143,5,c5,2
2,scaffold44|size37419_2,ID=44_2;partial=00;start_type=ATG;rbs_motif=No...,1108,2274,-,Strict,NaN,0.0,tetX,99.74,...,scaffold44|size37419_2#1109#2275#-1#ID=44_2;pa...,gnl|BL_ORD_ID|1398|hsp_num:0,776,cx_0004_0077_d9,Bacteroides_xylanisolvens,cx,4,77,d9,3
3,scaffold44|size37419_1,ID=44_1;partial=00;start_type=ATG;rbs_motif=No...,95,895,+,Strict,NaN,0.0,ErmF,99.25,...,scaffold44|size37419_1#96#896#1#ID=44_1;partia...,gnl|BL_ORD_ID|159|hsp_num:0,621,cx_0004_0077_d9,Bacteroides_xylanisolvens,cx,4,77,d9,3
4,scaffold13|size144083_82,ID=13_82;partial=00;start_type=ATG;rbs_motif=N...,110871,112796,+,Strict,NaN,0.0,tetQ,96.41,...,scaffold13|size144083_82#110872#112797#1#ID=13...,gnl|BL_ORD_ID|2201|hsp_num:0,1183,cx_0004_0077_d9,Bacteroides_xylanisolvens,cx,4,77,d9,3


Okay, first I want to just visualize the number of AMR genes per genome.

I have the tree from Mathieu's email uploaded into ITOL, so I need to create a gradient dataset annotation: https://itol.embl.de/help/dataset_gradient_template.txt

```
DATASET_GRADIENT
#In gradient datasets, each ID is associated to a single numeric value which is converted to a colored box based on the gradient defined.

#=================================================================#
#                    MANDATORY SETTINGS                           #
#=================================================================#
#select the separator which is used to delimit the data below (TAB,SPACE or COMMA).This separator must be used throught this file (except in the SEPARATOR line, which uses space).
SEPARATOR TAB

#label is used in the legend table (can be changed later)
DATASET_LABEL n_amr_genes_strict

#dataset color (can be changed later)
COLOR #ff0000

#=================================================================#
#                    OPTIONAL SETTINGS                            #
#=================================================================#

#each dataset can have a legend, which is defined below
#for each row in the legend, there should be one shape, color and label
#shape should be a number between 1 and 6:
#1: square
#2: circle
#3: star
#4: right pointing triangle
#5: left pointing triangle
#6: checkmark

#LEGEND_TITLE Dataset_legend
#LEGEND_SHAPES 1 2 3
#LEGEND_COLORS #ff0000 #00ff00 #0000ff
#LEGEND_LABELS value1 value2 value3

#=================================================================#
#     all other optional settings can be set or changed later     #
#           in the web interface (under 'Datasets' tab)           #
#=================================================================#

#width of the gradient strip
#STRIP_WIDTH 25

#left margin, used to increase/decrease the spacing to the next dataset. Can be negative, causing datasets to overlap.
#MARGIN 0

#border width; if set above 0, a border of specified width (in pixels) will be drawn around the gradient strip
#BORDER_WIDTH 0

#border color; used whern BORDER_WIDTH is above 0
#BORDER_COLOR #0000ff

#define the gradient colors. Values in the dataset will be mapped onto the corresponding color gradient.
#COLOR_MIN #ff0000
#COLOR_MAX #0000ff

#you can specify a gradient with three colors (e.g red to yellow to green) by setting 'USE_MID_COLOR' to 1, and specifying the midpoint color
#USE_MID_COLOR 1
#COLOR_MID #ffff00

#always show internal values; if set, values associated to internal nodes will be displayed even if these nodes are not collapsed. It could cause overlapping in the dataset display.
#SHOW_INTERNAL 1

#Internal tree nodes can be specified using IDs directly, or using the 'last common ancestor' method described in iTOL help pages
#=================================================================#
#       Actual data follows after the "DATA" keyword              #
#=================================================================#
DATA
#ID1 value1
#ID2 value2
#9606 10000
#LEAF1|LEAF2 11000
```

In [31]:
with open('../../data/analysis/itol_dataset.gradient.n_strict_hits.txt', 'w') as f:
    f.write('DATASET_GRADIENT\n')
    f.write('SEPARATOR TAB\n')
    f.write('DATASET_LABEL\tn_amr_genes_strict\n')
    f.write('COLOR\t#ff0000\n')
    
    # Min = white, max = black
    f.write('COLOR_MIN\t#ffffff\n')
    f.write('COLOR_MAX\t#000000\n')
    
    f.write('DATA\n')
    
    for i, j in zip(resdf['genome'], resdf['n_hits']):
        f.write(i + '\t' + str(j) + '\n')

In [50]:
resdf.sort_values(by='n_hits', ascending=False).groupby(['taxa', 'n_hits']).size().head()

taxa                   n_hits
Akkermansia_sp.        1           3
Alistipes_finegoldii   1           1
Alistipes_onderdonkii  1         265
                       8           8
Alistipes_shahii       1           1
dtype: int64

In [51]:
resdf[['genome', 'taxa', 'n_hits']].head()

,genome,taxa,n_hits
0,aa_0143_0005_c5,Bacteroides_ovatus,2
1,aa_0143_0005_c5,Bacteroides_ovatus,2
2,cx_0004_0077_d9,Bacteroides_xylanisolvens,3
3,cx_0004_0077_d9,Bacteroides_xylanisolvens,3
4,cx_0004_0077_d9,Bacteroides_xylanisolvens,3


In [52]:
# Total number of hits per taxa? 
# Did the tree already (need to color by phylogeny...)

# TODO: make a histogram

# Variation of number of hits per taxa?
# i.e. do most taxa tend to have the same number of ARGs, or do some vary a lot?

# Percent of genomes in each taxa that have at least one ARG
# This might be the same info as the variation...